In [1]:
import numpy as np
import os
import pandas as pd
import sys
from astropy.coordinates import SkyCoord
from astropy.io import fits
from astropy.nddata import Cutout2D
from astropy.nddata.utils import NoOverlapError
from astropy.units import arcsec
from astropy.wcs import WCS
from collections import OrderedDict

In [2]:
fitsdir = '../calibrated_files/fitsimg/'
csvdir = '../calibrated_files/'
df_orig = pd.read_csv('../reworked_fitting_code/final_data_files/kaluzny_2004_rrl_reformat.csv')

In [10]:
def make_irac_cutout(epoch,filt,df):
    base_name = 'omegaCen_{}_{}um'.format(epoch,filt)
    in_dataset = pd.read_csv(os.path.join(csvdir, base_name + '_rrl_matched_to_kaluzny.cut'))
    fitsfile, fitshead = fits.getdata(os.path.join(fitsdir, base_name + '_n.fits'),0,header=True)
    wcs = WCS(fitshead)
    size = 18*arcsec
    df_cut = df[df['id'].isin(in_dataset['ID'].str.split('V').str[-1].astype(int))]
    for index, row in df_cut.iterrows():
        cpath = 'cutouts/{}_{}.fits'.format(row.id, filt)
        position = SkyCoord(row.ra, row.dec, unit=('hour', 'deg'))
        try:
            cutout = Cutout2D(fitsfile, position, size, wcs=wcs, mode='partial', fill_value=0)
            cutout.data[cutout.data == np.nan] = 0
            fits.writeto(cpath, cutout.data, cutout.wcs.to_header(), clobber=True)
        except NoOverlapError:
            print row.id, 'wat'
#         except:
#             data = np.zeros((2,2))
#             fits.writeto(cpath, data)
#     for index, row in df.iterrows():
#         cpath = 'cutouts/{}_{}.fits'.format(row.ID.split('V')[-1], filt)
#         position = (row.xc_ref, row.yc_ref)
#         cutout = Cutout2D(fitsfile, position, size, wcs=wcs, mode='partial')
#         fits.writeto(cpath, cutout.data, cutout.wcs.to_header(), clobber=True)

In [11]:
for epoch in range(1,4):
    for filt in ['3p6','4p5']:
        make_irac_cutout(epoch,filt,df_orig)

In [9]:
def make_fourstar_cutout(filt,df):
    base_name = 'Omega_Cen_{}'.format(filt)
    data, head = fits.getdata(os.path.join(fitsdir, base_name + '.fits'),0,header=True)
    wcs = WCS(head)
    size = 18*arcsec
    for index, row in df.iterrows():
        cpath = 'cutouts/{}_{}.fits'.format(row.id, filt)
        position = SkyCoord(row.ra, row.dec, unit=('hour', 'deg'))
        try:
            cutout = Cutout2D(data, position, size, wcs=wcs, mode='partial', fill_value=0)
            cutout.data[cutout.data == np.nan] = 0
            fits.writeto(cpath, cutout.data, cutout.wcs.to_header(), clobber=True)
        except:
            print row.id, 'fucked up'

In [8]:
make_fourstar_cutout('Ks',df_orig)

the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]


159 fucked up
181 fucked up
182 fucked up
281 fucked up


In [ ]:
def make_blank_cutouts(df):
    for index, row in df.iterrows():
        cpath = 'cutouts/{}_Ks.fits'.format(row.id)
        if os.path.exists(cpath):
            for filt in ['3p6','4p5']:
                if not os.path.exists(cpath.replace('Ks',filt)):
                    data = np.zeros((2,2))
                    fits.writeto(cpath.replace('Ks',filt), data, clobber=True)

In [ ]:
make_blank_cutouts(df_orig)

In [ ]:
import montage_wrapper as m
import glob

In [ ]:
inlist = glob.glob(fitsdir+"*_?_?p?um.fits")
outlist = [i.replace('.fits','_n.fits') for i in inlist]

In [ ]:
m.reproject(inlist,outlist,north_aligned=True)